# Workflow 
---
This notebook is a summarised version of the essential steps needed to train our model & generate a prediction. 
We assume using method 2 (column version) in this notebook. 

What you need for this notebook: 
- Original dataset 

The notebook contains 3 section: 
1. Data parsing & preprocessing (+ feature engineering & selection)
2. Model training 
3. Prediction 

We are using XGBoost as the model here. 

## Section 1: Data Parsing & Preprocessing 

In [1]:
import json
import pandas as pd
import numpy as np

In [10]:
# !gzip -d dataset2.json.gz


In [12]:
data_list = []

with open('dataset1.json', 'r') as json_file:
    for line in json_file:
        data = json.loads(line)
        data_list.append(data)

In [3]:
def explore_dataset(data_list):
    results = []
    for data in data_list:
        trans_id, first = next(iter(data.items()))
        position, second = next(iter(first.items()))
        sequence, data = next(iter(second.items()))

        avg = np.mean(data, axis=0)
        med = np.median(data, axis=0)
        #smallest = np.min(data, axis=0)
        #largest = np.max(data, axis=0)
        std = np.std(data, axis=0)

        result = [trans_id, position, sequence] + list(avg)+ list(med) + list(std)
        results.append(result)

        #result=[trans_id, position, sequence]
        #results += list(avg)+ list(med) + list(smallest) + list(largest)+ list(std)

    return results

In [13]:
results = explore_dataset(data_list)
result_df = pd.DataFrame(results)

In [14]:
result_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,ENST00000000233,244,AAGACCA,0.008722,3.990545,123.090909,0.010218,6.716545,125.503030,0.006773,...,79.90,0.006961,2.043149,5.111862,0.007740,3.162649,2.971836,0.004178,2.649395,2.414444
1,ENST00000000233,261,CAAACTG,0.006292,2.849892,109.027711,0.007084,3.197289,107.190964,0.007168,...,94.15,0.003249,1.066933,3.038837,0.004254,1.428168,3.689105,0.005046,1.082745,2.752559
2,ENST00000000233,316,GAAACAG,0.007237,2.782710,105.440237,0.007050,3.463964,98.539053,0.007141,...,88.90,0.004005,1.437534,1.661621,0.004286,0.972003,3.335626,0.003814,0.777871,1.486479
3,ENST00000000233,332,AGAACAT,0.010299,6.042386,129.304348,0.008195,2.733641,97.320652,0.005806,...,89.90,0.005658,2.288102,2.673294,0.004195,0.943466,1.945091,0.003578,1.054379,3.112562
4,ENST00000000233,368,AGGACAA,0.011535,6.031751,118.000000,0.012108,5.748927,121.485876,0.009484,...,85.20,0.007941,1.345474,2.928527,0.005699,1.745973,2.321862,0.004775,1.206345,2.331742


In [15]:
colnames = ['transcript_id', 'transcript_position', 'sequence']
for i in ['avg', 'med','std']:
    for j in ['1-flank', 'central', '1+flank']:
        for q in ["length", "std", "mean"]:
            colnames.append(i + '_' + j + '_' + q)

result_df.columns = colnames

In [16]:
result_df['transcript_position'] = result_df['transcript_position'].astype(int)

In [17]:
result_df.head()

,transcript_id,transcript_position,sequence,avg_1-flank_length,avg_1-flank_std,avg_1-flank_mean,avg_central_length,avg_central_std,avg_central_mean,avg_1+flank_length,...,med_1+flank_mean,std_1-flank_length,std_1-flank_std,std_1-flank_mean,std_central_length,std_central_std,std_central_mean,std_1+flank_length,std_1+flank_std,std_1+flank_mean
0,ENST00000000233,244,AAGACCA,0.008722,3.990545,123.090909,0.010218,6.716545,125.503030,0.006773,...,79.90,0.006961,2.043149,5.111862,0.007740,3.162649,2.971836,0.004178,2.649395,2.414444
1,ENST00000000233,261,CAAACTG,0.006292,2.849892,109.027711,0.007084,3.197289,107.190964,0.007168,...,94.15,0.003249,1.066933,3.038837,0.004254,1.428168,3.689105,0.005046,1.082745,2.752559
2,ENST00000000233,316,GAAACAG,0.007237,2.782710,105.440237,0.007050,3.463964,98.539053,0.007141,...,88.90,0.004005,1.437534,1.661621,0.004286,0.972003,3.335626,0.003814,0.777871,1.486479
3,ENST00000000233,332,AGAACAT,0.010299,6.042386,129.304348,0.008195,2.733641,97.320652,0.005806,...,89.90,0.005658,2.288102,2.673294,0.004195,0.943466,1.945091,0.003578,1.054379,3.112562
4,ENST00000000233,368,AGGACAA,0.011535,6.031751,118.000000,0.012108,5.748927,121.485876,0.009484,...,85.20,0.007941,1.345474,2.928527,0.005699,1.745973,2.321862,0.004775,1.206345,2.331742


In [18]:
# feature selection 
df = result_df.copy() 
rfe = ['transcript_id',
'transcript_position',
'sequence',
'avg_central_mean',
'avg_1+flank_std',
'med_central_std',
'med_central_mean',
'med_1+flank_std',
'std_1-flank_std',
'std_1-flank_mean',
'std_central_std',
'std_central_mean',
'std_1+flank_std',
'std_1+flank_mean']
df = df[rfe]

In [19]:
# process sequential data
df['seq_left'] = df['sequence'].str[0:5]
df['seq_center'] = df['sequence'].str[1:6]
df['seq_right'] = df['sequence'].str[2:7]

In [21]:
# label encoding 
from sklearn.preprocessing import LabelEncoder
df_le = df.copy()
label_encoder = LabelEncoder()

seq_data = ['seq_left','seq_center','seq_right']
for seq in seq_data: 
    encoded_labels = label_encoder.fit_transform(df_le[seq])
    df_le[seq] = encoded_labels

df_le.head()

,transcript_id,transcript_position,sequence,avg_central_mean,avg_1+flank_std,med_central_std,med_central_mean,med_1+flank_std,std_1-flank_std,std_1-flank_mean,std_central_std,std_central_mean,std_1+flank_std,std_1+flank_mean,seq_left,seq_center,seq_right
0,ENST00000000233,244,AAGACCA,125.503030,3.999745,6.160,126.00,3.120,2.043149,5.111862,3.162649,2.971836,2.649395,2.414444,1,4,16
1,ENST00000000233,261,CAAACTG,107.190964,2.686892,3.020,108.00,2.540,1.066933,3.038837,1.428168,3.689105,1.082745,2.752559,6,2,10
2,ENST00000000233,316,GAAACAG,98.539053,2.083775,3.500,98.90,1.950,1.437534,1.661621,0.972003,3.335626,0.777871,1.486479,12,0,2
3,ENST00000000233,332,AGAACAT,97.320652,2.252565,2.565,97.35,2.175,2.288102,2.673294,0.943466,1.945091,1.054379,3.112562,2,6,3
4,ENST00000000233,368,AGGACAA,121.485876,3.958192,5.610,122.00,3.810,1.345474,2.928527,1.745973,2.321862,1.206345,2.331742,3,9,12


## Section 2: Model 

In [22]:
import xgboost as xgb

In [24]:
test = df_le.copy()

In [25]:
X_test = test.drop(['transcript_id', 'sequence'], axis=1)
X_test.head()

,transcript_position,avg_central_mean,avg_1+flank_std,med_central_std,med_central_mean,med_1+flank_std,std_1-flank_std,std_1-flank_mean,std_central_std,std_central_mean,std_1+flank_std,std_1+flank_mean,seq_left,seq_center,seq_right
0,244,125.503030,3.999745,6.160,126.00,3.120,2.043149,5.111862,3.162649,2.971836,2.649395,2.414444,1,4,16
1,261,107.190964,2.686892,3.020,108.00,2.540,1.066933,3.038837,1.428168,3.689105,1.082745,2.752559,6,2,10
2,316,98.539053,2.083775,3.500,98.90,1.950,1.437534,1.661621,0.972003,3.335626,0.777871,1.486479,12,0,2
3,332,97.320652,2.252565,2.565,97.35,2.175,2.288102,2.673294,0.943466,1.945091,1.054379,3.112562,2,6,3
4,368,121.485876,3.958192,5.610,122.00,3.810,1.345474,2.928527,1.745973,2.321862,1.206345,2.331742,3,9,12


In [26]:
# minmax normalisation 
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
d = scaler.fit_transform(X_test)
normalised_X_test = pd.DataFrame(d, columns=X_test.columns)
normalised_X_test.head()

,transcript_position,avg_central_mean,avg_1+flank_std,med_central_std,med_central_mean,med_1+flank_std,std_1-flank_std,std_1-flank_mean,std_central_std,std_central_mean,std_1+flank_std,std_1+flank_mean,seq_left,seq_center,seq_right
0,0.017035,0.784456,0.298511,0.439353,0.782178,0.208155,0.052962,0.301764,0.089098,0.266979,0.089200,0.077677,0.043478,0.235294,0.695652
1,0.018237,0.406697,0.156152,0.157233,0.425743,0.145923,0.024773,0.161799,0.037241,0.351434,0.032719,0.091367,0.260870,0.117647,0.434783
2,0.022125,0.228217,0.090752,0.200359,0.245545,0.082618,0.035474,0.068814,0.023603,0.309813,0.021727,0.040106,0.521739,0.000000,0.086957
3,0.023256,0.203082,0.109055,0.116352,0.214851,0.106760,0.060035,0.137119,0.022750,0.146085,0.031696,0.105942,0.086957,0.352941,0.130435
4,0.025801,0.701586,0.294006,0.389937,0.702970,0.282189,0.032816,0.154352,0.046743,0.190448,0.037175,0.074329,0.130435,0.529412,0.521739


In [28]:
import joblib
# Load the trained model from the file
best_model = joblib.load('method2_columns/bext_xgboost.pkl')

In [29]:
test.head()

,transcript_id,transcript_position,sequence,avg_central_mean,avg_1+flank_std,med_central_std,med_central_mean,med_1+flank_std,std_1-flank_std,std_1-flank_mean,std_central_std,std_central_mean,std_1+flank_std,std_1+flank_mean,seq_left,seq_center,seq_right
0,ENST00000000233,244,AAGACCA,125.503030,3.999745,6.160,126.00,3.120,2.043149,5.111862,3.162649,2.971836,2.649395,2.414444,1,4,16
1,ENST00000000233,261,CAAACTG,107.190964,2.686892,3.020,108.00,2.540,1.066933,3.038837,1.428168,3.689105,1.082745,2.752559,6,2,10
2,ENST00000000233,316,GAAACAG,98.539053,2.083775,3.500,98.90,1.950,1.437534,1.661621,0.972003,3.335626,0.777871,1.486479,12,0,2
3,ENST00000000233,332,AGAACAT,97.320652,2.252565,2.565,97.35,2.175,2.288102,2.673294,0.943466,1.945091,1.054379,3.112562,2,6,3
4,ENST00000000233,368,AGGACAA,121.485876,3.958192,5.610,122.00,3.810,1.345474,2.928527,1.745973,2.321862,1.206345,2.331742,3,9,12


In [30]:
prob = best_model.predict_proba(normalised_X_test)[:,1]
transcript_id = test['transcript_id']
transcript_position = test['transcript_position']
data = {
    'transcript_id': transcript_id,
    'transcript_position': transcript_position,
    'probability': prob
}

# Create a DataFrame from the data dictionary
output = pd.DataFrame(data)

output.head()



,transcript_id,transcript_position,probability
0,ENST00000000233,244,0.048335
1,ENST00000000233,261,0.073142
2,ENST00000000233,316,0.011085
3,ENST00000000233,332,0.049029
4,ENST00000000233,368,0.011907


In [ ]:
# export output
file_name = 'tundra_dataset1_1.csv'
output.to_csv(file_name, index=False)